In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Kural Tabanlı Sınıflandırma ile
# Potansiyel Müşteri Getirisi Hesaplama
*İş Problemi*
> Bir oyun şirketi müşterilerinin bazı özelliklerini kullanarak
seviye tabanlı (level based) yeni müşteri tanımları (persona)
oluşturmak ve bu yeni müşteri tanımlarına göre segmentler
oluşturup bu segmentlere göre yeni gelebilecek müşterilerin
şirkete ortalama ne kadar kazandırabileceğini tahmin etmek
istemektedir.
Örneğin:
Türkiye’den IOS kullanıcısı olan 25 yaşındaki bir erkek
kullanıcının ortalama ne kadar kazandırabileceği belirlenmek
isteniyor.

*Veri Seti Hikayesi*
> Persona.csv veri seti uluslararası bir oyun şirketinin sattığı ürünlerin fiyatlarını ve bu
> ürünleri satın alan kullanıcıların bazı demografik bilgilerini barındırmaktadır. Veri
> seti her satış işleminde oluşan kayıtlardan meydana gelmektedir. Bunun anlamı
> tablo tekilleştirilmemiştir. Diğer bir ifade ile belirli demografik özelliklere sahip bir
> kullanıcı birden fazla alışveriş yapmış olabilir.

In [2]:
import numpy as np
import pandas as pd 
import seaborn as sns

In [3]:
#Soru 1: persona.csv dosyasını okutalım 
df = pd.read_csv("/kaggle/input/personacsv/persona.csv")

In [4]:
#Veri seti ile ilgili genel bilgiler
# Price: Müşterinin harcama tutarı
# Source: Müşterinin bağlandığı cihaz türü
# Sex: Müşterinin cinsiyeti
# Country: Müşterinin ülkesi
# Age: Müşterinin yaşı
df.head()
df.shape

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


(5000, 5)

In [5]:
# Kaç unique SOURCE vardır ve fekansları nedir inceleyelim
df["SOURCE"].nunique()
df["SOURCE"].value_counts()

2

SOURCE
android    2974
ios        2026
Name: count, dtype: int64

In [6]:
#  Kaç unique PRICE vardır ve hangi PRICE'dan kaçar tane satış gerçekleşmiştir inceleyelim
df["PRICE"].nunique()
df["PRICE"].value_counts()

6

PRICE
29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: count, dtype: int64

In [7]:
#Hangi ülkeden kaçar tane satış olmuş bir bakalım
df["COUNTRY"].value_counts()

COUNTRY
usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: count, dtype: int64

In [8]:
#  Ülkelere göre satışlardan toplam ne kadar kazanılmış inceleyelim
df.groupby("COUNTRY")["PRICE"].sum()

COUNTRY
bra    51354
can     7730
deu    15485
fra    10177
tur    15689
usa    70225
Name: PRICE, dtype: int64

In [9]:
#  Ülkelere göre PRICE ortalamalarını ve 
df.groupby(by=['COUNTRY']).agg({"PRICE": "mean"})

# SOURCE'lara göre PRICE ortalamalarına bakalım
df.groupby(by=['SOURCE']).agg({"PRICE": "mean"})

,PRICE
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


,PRICE
SOURCE,
android,34.174849
ios,34.069102


In [10]:
# COUNTRY ve SOURCE'a göre PRICE ortalamaları nedir inceleyelim
df.groupby(by=["COUNTRY", 'SOURCE']).agg({"PRICE": "mean"})

PRICE
COUNTRY SOURCE            
bra     android  34.387029
        ios      34.222222
can     android  33.330709
        ios      33.951456
deu     android  33.869888
        ios      34.268817
fra     android  34.312500
        ios      32.776224
tur     android  36.229437
        ios      33.272727
usa     android  33.760357
        ios      34.371703

In [11]:
#Adım1:COUNTRY, SOURCE, SEX, AGE kırılımında ortalama kazançlar nedir?
df.groupby(["COUNTRY", 'SOURCE', "SEX", "AGE"]).agg({"PRICE": "mean"}).head()

PRICE
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897

In [12]:
# Adım2:Çıktıyı PRICE'a göre sıralayınız.
# Önceki sorudaki çıktıyı daha iyi görebilmek için sort_values metodunu azalan olacak şekilde PRICE'a uygulayınız.
# Çıktıyı agg_df olarak kaydediniz.
agg_df = df.groupby(by=["COUNTRY", 'SOURCE', "SEX", "AGE"]).agg({"PRICE": "mean"}).sort_values("PRICE", ascending=False)
agg_df.head()

,,,,PRICE
COUNTRY,SOURCE,SEX,AGE,
bra,android,male,46,59.0
usa,android,male,36,59.0
fra,android,female,24,59.0
usa,ios,male,32,54.0
deu,android,female,36,49.0


In [13]:
# Adım3: Indekste yer alan isimleri değişken ismine çeviriniz.
# Üçüncü sorunun çıktısında yer alan PRICE dışındaki tüm değişkenler index isimleridir.
# Bu isimleri değişken isimlerine çeviriniz.
agg_df = agg_df.reset_index()
agg_df.head()
agg_df.columns


,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0


Index(['COUNTRY', 'SOURCE', 'SEX', 'AGE', 'PRICE'], dtype='object')

In [14]:
# Adım4: AGE değişkenini kategorik değişkene çeviriniz ve agg_df'e ekleyiniz.
# Age sayısal değişkenini kategorik değişkene çeviriniz.
# Örneğin: '0_18', '19_23', '24_30', '31_40', '41_70'

# AGE değişkeninin bölüneceği yerler:
bins = [0, 18, 23, 30, 40, agg_df["AGE"].max()]

# Bölünen noktalara karşılık gelen isimlendirmeler:
etiket = ['0_18', '19_23', '24_30', '31_40', '41_' + str(agg_df["AGE"].max())] #neden str?

# age'i bölelim:
agg_df["AGE_CAT"] = pd.cut(agg_df["AGE"], bins, labels=etiket)
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT
0,bra,android,male,46,59.0,41_66
1,usa,android,male,36,59.0,31_40
2,fra,android,female,24,59.0,24_30
3,usa,ios,male,32,54.0,31_40
4,deu,android,female,36,49.0,31_40


In [15]:
#Adım5:Yeni level based müşterileri tanımlayınız ve veri setine değişken olarak ekleyiniz.
# customers_level_based adında bir değişken tanımlayınız ve veri setine bu değişkeni ekleyiniz.
agg_df["CUSTOMER_LEVEL_BASED"] = [row[0].upper() + "_" + row[1].upper() + "_" + row[2].upper() + "_" + row[5].upper() for row in agg_df.values]
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,CUSTOMER_LEVEL_BASED
0,bra,android,male,46,59.0,41_66,BRA_ANDROID_MALE_41_66
1,usa,android,male,36,59.0,31_40,USA_ANDROID_MALE_31_40
2,fra,android,female,24,59.0,24_30,FRA_ANDROID_FEMALE_24_30
3,usa,ios,male,32,54.0,31_40,USA_IOS_MALE_31_40
4,deu,android,female,36,49.0,31_40,DEU_ANDROID_FEMALE_31_40


In [16]:
# list comp ile customers_level_based değerleri oluşturulduktan sonra bu değerlerin tekilleştirilmesi gerekmektedir.
# Örneğin birden fazla şu ifadeden olabilir: USA_ANDROID_MALE_0_18
# Bunları groupby'a alıp price ortalamalarını almak gerekmektedir.
agg_df = agg_df.groupby("CUSTOMER_LEVEL_BASED").agg({"PRICE": "mean"})
agg_df.columns

Index(['PRICE'], dtype='object')

In [17]:
# CUSTOMER_LEVEL_BASED index'te yer almaktadır.Bunu değişkene çevirelim.
agg_df = agg_df.reset_index()
agg_df.head()

,CUSTOMER_LEVEL_BASED,PRICE
0,BRA_ANDROID_FEMALE_0_18,35.645303
1,BRA_ANDROID_FEMALE_19_23,34.077340
2,BRA_ANDROID_FEMALE_24_30,33.863946
3,BRA_ANDROID_FEMALE_31_40,34.898326
4,BRA_ANDROID_FEMALE_41_66,36.737179


In [18]:
# Adım 6: Yeni müşterileri (USA_ANDROID_MALE_0_18) segmentlere ayırınız.
# PRICE'a göre segmentlere ayırınız,
# segmentleri "SEGMENT" isimlendirmesi ile agg_df'e ekleyiniz,
# segmentleri betimleyiniz,
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels=["D", "C", "B", "A"])
agg_df.head(10)
agg_df.groupby("SEGMENT", observed=True )["PRICE"].mean()

,CUSTOMER_LEVEL_BASED,PRICE,SEGMENT
0,BRA_ANDROID_FEMALE_0_18,35.645303,B
1,BRA_ANDROID_FEMALE_19_23,34.077340,C
2,BRA_ANDROID_FEMALE_24_30,33.863946,C
3,BRA_ANDROID_FEMALE_31_40,34.898326,B
4,BRA_ANDROID_FEMALE_41_66,36.737179,A
5,BRA_ANDROID_MALE_0_18,34.805861,B
6,BRA_ANDROID_MALE_19_23,31.673243,D
7,BRA_ANDROID_MALE_24_30,33.413919,C
8,BRA_ANDROID_MALE_31_40,34.327381,B
9,BRA_ANDROID_MALE_41_66,40.041667,A


SEGMENT
D    29.206780
C    33.509674
B    34.999645
A    38.691234
Name: PRICE, dtype: float64

In [19]:
# Adım 7: Yeni gelen müşterileri sınıflandırınız ne kadar gelir getirebileceğini tahmin ediniz.
# 33 yaşında ANDROID kullanan bir Türk kadını hangi segmente aittir ve ortalama ne kadar gelir kazandırması beklenir?
new_user = "TUR_ANDROID_FEMALE_31_40"
agg_df[agg_df["CUSTOMER_LEVEL_BASED"] == new_user]

,CUSTOMER_LEVEL_BASED,PRICE,SEGMENT
72,TUR_ANDROID_FEMALE_31_40,41.833333,A


In [20]:
 # 35 yaşında IOS kullanan bir Fransız kadını hangi segmente ve ortalama ne kadar gelir kazandırması beklenir?
new_user = "FRA_IOS_FEMALE_31_40"
agg_df[agg_df["CUSTOMER_LEVEL_BASED"] == new_user]

,CUSTOMER_LEVEL_BASED,PRICE,SEGMENT
63,FRA_IOS_FEMALE_31_40,32.818182,C
